In [ ]:
from google.colab import files
uploaded = files.upload()

Saving train_test_networkP.csv to train_test_networkP.csv


In [ ]:
import pandas as pd

# Load the dataset
file_name = "train_test_networkP.csv"
df = pd.read_csv(file_name)

# Display dataset shape and columns
print("Dataset Shape:", df.shape)
print("Columns:", df.columns)

Dataset Shape: (211043, 31)
Columns: Index(['proto', 'service', 'duration', 'src_bytes', 'dst_bytes', 'conn_state',
       'missed_bytes', 'src_pkts', 'src_ip_bytes', 'dst_pkts', 'dst_ip_bytes',
       'dns_query', 'dns_qclass', 'dns_qtype', 'dns_rcode', 'dns_AA', 'dns_RD',
       'dns_RA', 'dns_rejected', 'ssl_version', 'ssl_cipher', 'ssl_resumed',
       'ssl_established', 'ssl_subject', 'http_trans_depth', 'http_method',
       'http_version', 'http_request_body_len', 'http_response_body_len',
       'http_status_code', 'Label'],
      dtype='object')


In [ ]:

import numpy as np

# Set a random seed for reproducibility
np.random.seed(42)

# Shuffle the dataset
df = df.sample(frac=1, random_state=42).reset_index(drop=True)

# Split Labeled (5%) and Unlabeled (95%)
labeled_df = df.sample(frac=0.05, random_state=42)
unlabeled_df = df.drop(labeled_df.index)

# Display the split
print("Labeled Data Shape:", labeled_df.shape)
print("Unlabeled Data Shape:", unlabeled_df.shape)


Labeled Data Shape: (10552, 31)
Unlabeled Data Shape: (200491, 31)


In [ ]:
from sklearn.model_selection import train_test_split

label_column = "Label"

# Drop labels from the Unlabeled Dataset
unlabeled_df_no_label = unlabeled_df.drop(columns=[label_column])

# Further split Unlabeled Data (Train 90% / Test 10%)
unlabeled_train_df, unlabeled_test_df = train_test_split(unlabeled_df_no_label, test_size=0.10, random_state=42)

# Display updated split sizes
print("Unlabeled Training Data Shape (without labels):", unlabeled_train_df.shape)
print("Unlabeled Testing Data Shape (without labels):", unlabeled_test_df.shape)


Unlabeled Training Data Shape (without labels): (180441, 30)
Unlabeled Testing Data Shape (without labels): (20050, 30)


In [ ]:
# Save datasets
labeled_df.to_csv("labeled_data.csv", index=False)
print("Labeled dataset saved as 'labeled_data.csv'")

unlabeled_train_df.to_csv("unlabeled_train_data.csv", index=False)
print("Unlabeled train dataset saved as 'unlabeled_train_data.csv'")

unlabeled_test_df.to_csv("unlabeled_test_data.csv", index=False)
print("Unlabeled test dataset saved as 'unlabeled_test_data.csv'")

Labeled dataset saved as 'labeled_data.csv'
Unlabeled train dataset saved as 'unlabeled_train_data.csv'
Unlabeled test dataset saved as 'unlabeled_test_data.csv'


In [ ]:
import pandas as pd
from sklearn.model_selection import train_test_split

# Load the labeled dataset
labeled_df = pd.read_csv("labeled_data.csv")

# Define features (X) and labels (y)
label_column = "Label"
X = labeled_df.drop(columns=[label_column])  # Features
y = labeled_df[label_column]  # Labels

# Split into 80% Training and 20% Validation
X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.20, random_state=42)

# Display dataset shapes
print("Training Data Shape:", X_train.shape)
print("Validation Data Shape:", X_val.shape)


Training Data Shape: (8441, 30)
Validation Data Shape: (2111, 30)


In [ ]:
from sklearn.ensemble import RandomForestClassifier, BaggingClassifier, AdaBoostClassifier
from sklearn.tree import DecisionTreeClassifier
from xgboost import XGBClassifier
from sklearn.metrics import accuracy_score

# Define models
models = {
    "XGBoost": XGBClassifier(eval_metric='logloss'),
    "Random Forest": RandomForestClassifier(n_estimators=100, random_state=42),
    "Decision Tree": DecisionTreeClassifier(random_state=42),
    "Bagging" : BaggingClassifier(estimator=DecisionTreeClassifier(), n_estimators=50, random_state=42),
    "AdaBoost": AdaBoostClassifier(n_estimators=50, random_state=42)
}

# Train each model and evaluate performance
for name, model in models.items():
    model.fit(X_train, y_train)  # Train the model
    y_pred = model.predict(X_val)  # Predict on validation set
    acc = accuracy_score(y_val, y_pred)  # Calculate accuracy
    print(f"{name} Accuracy: {acc:.4f}")


XGBoost Accuracy: 0.9967
Random Forest Accuracy: 0.9972
Decision Tree Accuracy: 0.9943
Bagging Accuracy: 0.9976
AdaBoost Accuracy: 0.9848
